## Example Inference

### Connect KL520

In [1]:
import cv2
import kp

In [2]:
device_descriptors = kp.core.scan_devices()

if 0 < device_descriptors.device_descriptor_number:
    usb_port_id = device_descriptors.device_descriptor_list[0].usb_port_id
    print(usb_port_id)
else:
    print('Error: no Kneron device connect.')
    exit(0)

13


In [3]:
device_group = kp.core.connect_devices(usb_port_ids=[22])

In [4]:
kp.core.set_timeout(device_group=device_group,
                    milliseconds=5000)

### Load firmware, model, and test image

In [5]:
SCPU_FW_PATH = '../../res/firmware/KL520/fw_scpu.bin'
NCPU_FW_PATH = '../../res/firmware/KL520/fw_ncpu.bin'
kp.core.load_firmware_from_file(device_group=device_group,
                                scpu_fw_path=SCPU_FW_PATH,
                                ncpu_fw_path=NCPU_FW_PATH)

In [6]:
MODEL_FILE_PATH = './declutter_520.nef'
model_nef_descriptor = kp.core.load_model_from_file(device_group=device_group,
                                                    file_path=MODEL_FILE_PATH)

### pre-processing

In [7]:
IMAGE_FILE_PATH = './test.jpg'

img = cv2.imread(filename=IMAGE_FILE_PATH)
img_bgr565 = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2BGR565)

In [8]:
generic_inference_input_descriptor = kp.GenericImageInferenceDescriptor(
    model_id=model_nef_descriptor.models[0].id,
    inference_number=0,
    input_node_image_list=[
        kp.GenericInputNodeImage(
            image=img_bgr565,
            image_format=kp.ImageFormat.KP_IMAGE_FORMAT_RGB565,
            resize_mode=kp.ResizeMode.KP_RESIZE_ENABLE,
            padding_mode=kp.PaddingMode.KP_PADDING_CORNER,
            normalize_mode=kp.NormalizeMode.KP_NORMALIZE_KNERON
        )
    ]
)

In [9]:
kp.inference.generic_image_inference_send(device_group=device_group,
                                          generic_inference_input_descriptor=generic_inference_input_descriptor)
generic_raw_result = kp.inference.generic_image_inference_receive(device_group=device_group)

In [10]:
inf_node_output_list = []

for node_idx in range(generic_raw_result.header.num_output_node):
    inference_float_node_output = kp.inference.generic_inference_retrieve_float_node(node_idx=node_idx,
                                                                                     generic_raw_result=generic_raw_result,
                                                                                     channels_ordering=kp.ChannelOrdering.KP_CHANNEL_ORDERING_CHW)
    inf_node_output_list.append(inference_float_node_output)

print(inf_node_output_list)

[{
    "width": 1,
    "height": 1,
    "channel": 512,
    "channels_ordering": "ChannelOrdering.KP_CHANNEL_ORDERING_CHW",
    "num_data": 512,
    "ndarray": [
        "[[[[0.5060123 ]]",
        "",
        "  [[0.        ]]",
        "",
        "  [[0.30360737]]",
        "",
        "  [[0.5060123 ]]",
        "",
        "  [[0.25300616]]",
        "",
        "  [[0.        ]]",
        "",
        "  [[0.05060123]]",
        "",
        "  [[1.113227  ]]",
        "",
        "  [[1.1638283 ]]",
        "",
        "  [[0.05060123]]",
        "",
        "  [[0.30360737]]",
        "",
        "  [[0.10120246]]",
        "",
        "  [[0.5060123 ]]",
        "",
        "  [[0.20240492]]",
        "",
        "  [[0.05060123]]",
        "",
        "  [[0.5566135 ]]",
        "",
        "  [[0.05060123]]",
        "",
        "  [[0.25300616]]",
        "",
        "  [[0.25300616]]",
        "",
        "  [[0.10120246]]",
        "",
        "  [[0.05060123]]",
        ""

### post-processing

In [22]:
import numpy as np
import re

data = inf_node_output_list[0]

raw_ndarray = data.ndarray

if isinstance(raw_ndarray, np.ndarray):
    ndarray_np = raw_ndarray
else:
    ndarray_np = np.array(raw_ndarray)

ndarray_np = ndarray_np.flatten()  
# ndarray_np = ndarray_np.reshape((data.channel, data.height, data.width))

print(f"dimension of feature tensor: {ndarray_np.size}\n")
print(ndarray_np)

dimension of feature tensor: 512

[0.5060123  0.         0.30360737 0.5060123  0.25300616 0.
 0.05060123 1.113227   1.1638283  0.05060123 0.30360737 0.10120246
 0.5060123  0.20240492 0.05060123 0.5566135  0.05060123 0.25300616
 0.25300616 0.10120246 0.05060123 0.7590185  0.5566135  0.
 0.05060123 0.15180369 0.30360737 0.7590185  0.657816   0.05060123
 0.         0.5566135  0.05060123 0.25300616 0.         0.05060123
 0.35420862 0.05060123 2.3782578  0.60721475 0.10120246 0.10120246
 0.30360737 0.70841724 0.45541108 0.35420862 0.25300616 0.30360737
 0.25300616 0.05060123 0.657816   0.05060123 0.45541108 0.10120246
 0.15180369 0.15180369 0.657816   0.25300616 0.         0.10120246
 0.         0.20240492 0.20240492 0.40480983 0.80961967 0.25300616
 0.25300616 0.05060123 0.05060123 0.         0.05060123 0.45541108
 0.45541108 0.40480983 0.10120246 0.         1.4674357  0.60721475
 0.10120246 0.15180369 0.15180369 0.7590185  0.45541108 0.
 0.         0.35420862 0.05060123 0.5566135  0.15180